# 01 · Exploración inicial de `Tab_Clientes(2)`

Fecha: 2025-08-10

Objetivo: entender estructura, nulos y valores para definir variables del clustering.


In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os, pandas as pd
import numpy as np

load_dotenv()

# Carpeta raíz del proyecto (sube un nivel desde /notebooks)
ROOT = Path.cwd().parent
DEFAULT = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"

# Permite override con .env si algún día quieres mover la ruta
DATA_PATH = Path(os.getenv("DATA_PATH", str(DEFAULT)))

import os
os.environ.pop("DATA_PATH", None)  # borra cualquier override

print("Usando CSV en:", DATA_PATH)
assert DATA_PATH.exists(), f"No encuentro el CSV en: {DATA_PATH}\n" \
                           f"Revisa que esté en data/raw o define DATA_PATH en un .env"

df = pd.read_csv(DATA_PATH, low_memory=False)
pd.set_option("display.max_columns", None)  # no truncar columnas
pd.set_option("display.width", None) 
df.shape, df.columns[:5].tolist()

In [ ]:
df.head(2)

In [ ]:
resumen = pd.DataFrame({
    'Tipo': df.dtypes.astype(str),
    'Nulos': df.isna().sum(),
    '%Nulos': (df.isna().sum()/len(df)*100).round(2)
}).sort_values('%Nulos', ascending=False)
resumen.head(20)


## Identificadores y duplicados
### Objetivo: validar si RutBeneficiario identifica único a cada fila o si hay duplicados (mismo beneficiario con múltiples registros).

In [ ]:
id_col = "RutBeneficiario"
n_total = len(df)
n_unicos = df[id_col].nunique(dropna=True)
dupes = (
    df[id_col]
    .value_counts(dropna=True)
    .loc[lambda s: s>1]
)

print("Filas totales:", n_total)
print("IDs únicos:", n_unicos)
print("Duplicados (IDs con >1 fila):", len(dupes))
dupes.head(10)


## Fechas (detección y rangos)
### Objetivo: ver cuáles columnas parsean bien como fecha y sus rangos.

In [ ]:
# 1) Solo columnas de texto
obj_cols = df.columns[df.dtypes == "object"].tolist()

# 2) Candidatas por nombre (fechas reales)
posibles_fechas = ["Fec_Nac","FechaRegistro","FechaUltimaVisita","FecUltimoPptoCreado","PrimeraFechaAtencion"]
candidatas_fecha = [c for c in posibles_fechas if c in obj_cols]

# 3) Parseo controlado (ojo con dayfirst)
parsed = {c: pd.to_datetime(df[c], errors="coerce", dayfirst=True) for c in candidatas_fecha}

# 4) % de parseo y rangos
pct = {c: parsed[c].notna().mean() for c in candidatas_fecha}
rangos = {c: (parsed[c].min(), parsed[c].max()) for c in candidatas_fecha if parsed[c].notna().any()}
pct, rangos



## Validar “días desde última visita”
### Objetivo: comprobar si DiasDesdeUltimaVisita coincide con FechaUltimaVisita (cuando existe).

In [ ]:
# --- Validación de DiasDesdeUltimaVisita vs. cálculo propio ---
import pandas as pd

# 1) Base de trabajo: si ya creaste df_exp (con fechas parseadas), úsalo; si no, usa df.
base = df_exp if 'df_exp' in locals() else df

# 2) Parseo de FechaUltimaVisita como ISO -> dayfirst=False (evita warnings, interpreta bien YYYY-MM-DD)
fuv = pd.to_datetime(base.get("FechaUltimaVisita"), errors="coerce", dayfirst=False)

# 3) "Fecha de corte observada": la máxima fecha en la columna (solo para comparar).
corte_obs = fuv.max()

# 4) Cálculo nuestro de días: diferencia entre corte observado y cada FechaUltimaVisita.
calc_dias = (corte_obs - fuv).dt.days

# 5) Columna real del sistema a numérico (maneja nulos/strings).
real = pd.to_numeric(base.get("DiasDesdeUltimaVisita"), errors="coerce")

# 6) Filas comparables: donde ambas series tienen dato.
mask = fuv.notna() & real.notna()

# 7) Diferencia con signo: real (sistema) - calculado (nuestro)
#    Si es >0, el sistema suele contar con un "corte" más adelante.
diff = (real[mask] - calc_dias[mask])

# 8) Métricas principales del desfase
print("Filas comparables:", int(mask.sum()))
print("Mediana diferencia (días):", float(diff.median()))
print("P90 diferencia:", float(diff.quantile(0.90)))
print("Proporción |dif| <= 7 días:", float((diff.abs() <= 7).mean()))

# 9) Dirección del desfase (distribución de signos)
signo = diff.map(lambda x: 1 if x>0 else (-1 if x<0 else 0))
print("Signo de la diferencia → proporciones:", signo.value_counts(normalize=True).to_dict())

# 10) “Offset documental”: lo dejamos anotado para Notion (no lo usamos en el modelo)
offset_documental = float(diff.median())
print("Offset documental sugerido:", int(round(offset_documental)), "días")

# 11) Decisión operativa explícita
usar_columna_sistema = True
print("Usaremos DiasDesdeUltimaVisita del sistema:", usar_columna_sistema)



## Presupuestos (calidad y outliers)
### Objetivo: ver nulos/ceros y magnitudes para tomar decisiones de limpieza (y luego derivar KPIs en el 02).

In [ ]:
# ── 0) Base de trabajo: si ya tienes df_exp (con fechas parseadas/flags), úsalo. Si no, df.
base = df_exp if 'df_exp' in locals() else df

# ── 1) Definimos las columnas de presupuestos (cuentas y montos)
cols_p = ["CantPptos","TotPptos","CantPptosAbo","TotPptosAbo","CantPptosAvan","TotPptosAvan"]
cols_p = [c for c in cols_p if c in base.columns]  # por si falta alguna

# ── 2) Tasa de nulos y de ceros (calidad y “ausencia” de actividad)
nulos = base[cols_p].isna().mean().round(3)
ceros = (base[cols_p] == 0).mean().round(3)

# ── 3) Búsqueda de valores negativos (deberían ser 0 en clínicas; si aparecen, son errores)
negativos = (base[cols_p] < 0).sum()

# ── 4) Estadísticos y percentiles altos (cola larga = posible outlier)
stat = base[cols_p].describe(percentiles=[.5,.9,.95,.99]).T

# ── 5) ¿Quiénes “tienen presupuestos”? (al menos una de las columnas no nula)
tiene_ppto = base[cols_p].notna().any(axis=1)
prop_tiene_ppto = tiene_ppto.mean().round(3)

# ── 6) Resumen compacto para leer en una tabla
resumen = pd.concat([
    nulos.rename("pct_nulos"),
    ceros.rename("pct_ceros"),
    negativos.rename("n_negativos")
], axis=1).join(stat)

print("Filas totales:", len(base))
print("Proporción con algún dato de presupuestos:", prop_tiene_ppto)
resumen



## Ventanas de atención (cobertura y correlación)
### Objetivo: ver en cuáles ventanas hay más actividad y qué tan redundantes son.

In [ ]:
# ── 0) Base: usamos df_exp si existe; si no, df.
base = df_exp if 'df_exp' in locals() else df

# ── 1) Detectamos TODAS las columnas que empiezan por "Atencion"
#     (sin depender de mayúsculas; así no nos saltamos ninguna)
att_cols = [c for c in base.columns if c.lower().startswith("atencion")]
print("Columnas de atención detectadas:", att_cols, "\n")

# ── 2) Cobertura: ¿qué % de filas tiene valor > 0 en cada ventana?
#     (Si alguien tuvo al menos 1 atención en la ventana, cuenta como 1)
cover = (
    base[att_cols]
    .fillna(0)
    .gt(0)                 # True si >0
    .mean()                # promedio → % de filas con True
    .sort_values(ascending=False)
    .round(4)
)
print("Cobertura (>0) por ventana (ordenadas):\n", cover, "\n")

# ── 3) Marcamos columnas que NO aportan (cobertura 0 → siempre vacías)
drop_zero = cover[cover == 0].index.tolist()
print("Candidatas a descartar por cobertura 0:", drop_zero, "\n")

# ── 4) Redundancia: correlación entre ventanas (¿cuáles se mueven juntas?)
#     Si dos columnas están muy correlacionadas (>= 0.9), básicamente cuentan lo mismo.
X = base[att_cols].fillna(0)
corr = X.corr()

pairs = (
    corr.where(~np.eye(corr.shape[0], dtype=bool))  # anulamos diagonal
        .stack()
        .rename("corr")
        .sort_values(ascending=False)
)

high = pairs[pairs.abs() >= 0.90]   # umbral de "muy similares"
print("Pares muy correlacionados (|r| >= 0.90):\n", high.head(10), "\n")

# ── 5) Selección automática (greedy): 
#     vamos agregando ventanas de mayor cobertura, 
#     pero saltamos las que están muy correlacionadas (> 0.9) con alguna ya elegida.
seleccion = []
thr = 0.90
for c in cover.index:
    if c in drop_zero:
        continue
    if all(abs(corr.loc[c, s]) < thr for s in seleccion):
        seleccion.append(c)

print("Selección greedy (sin colinealidad alta):", seleccion, "\n")

# ── 6) Propuesta humana (simple y entendible):
#     cubrir todo el eje temporal con pocas ventanas y sin redundancia clara.
propuesta_humana = [c for c in ["Atencion15d","Atencion1m","Atencion3m","Atencion6m","Atencion1a","Atencion2a"] if c in att_cols]
print("Propuesta humana:", propuesta_humana)



In [ ]:
# Base de trabajo
base = df_exp if 'df_exp' in locals() else df

# Selección acordada
att_selected = [c for c in ["Atencion15d","Atencion1m","Atencion3m","Atencion6m","Atencion1a","Atencion2a"] if c in base.columns]

# Creamos variables de presencia (0/1)
df_att = base.copy()
for c in att_selected:
    df_att[c + "_pres"] = (df_att[c].fillna(0) > 0).astype(int)

# Resumen de cobertura de las nuevas columnas
pres_cols = [c + "_pres" for c in att_selected]
coverage_pres = df_att[pres_cols].mean().sort_values(ascending=False).round(4)
print("Features de presencia creadas:", pres_cols)
print("\nCobertura (proporción=1) por ventana seleccionada:\n", coverage_pres)

# (Opcional) guardamos un intermedio para revisar luego
from pathlib import Path
Path("../data/interim").mkdir(parents=True, exist_ok=True)
df_att[pres_cols].to_csv("../data/interim/atencion_presencia_v1.csv", index=False)
print("\nExportado: ../data/interim/atencion_presencia_v1.csv")


## Geografía (cardinalidad)
### Objetivo: ver si conviene usar Comuna tal cual (mucha cardinalidad) o agrupar/usar Región.

In [ ]:
# ── 0) Base de trabajo: usamos df_exp si existe (ya con fechas/flags), si no df
base = df_exp if 'df_exp' in locals() else df

# ── 1) Elegimos N (cuántas comunas dejaremos "tal cual")
TOP_N = 20  # puedes cambiarlo luego si te parece mucho/poco

# ── 2) Conteo de comunas para ver las más frecuentes
comunas_count = (
    base["Comuna"]
    .fillna("Sin Comuna")    # homogeneizamos nulos
    .astype(str)
    .value_counts()
)

# ── 3) Construimos una versión agrupada: Top-N, "Sin Comuna" explícito, resto a Otras/Infreq
top_comunas = comunas_count.head(TOP_N).index
def agrupar_comuna(x):
    x = "Sin Comuna" if pd.isna(x) or str(x).strip()=="" else str(x)
    if x == "Sin Comuna":
        return "Sin Comuna"
    return x if x in top_comunas else "Otras/Infreq"

df_geo = base.copy()
df_geo["Comuna_grp"] = df_geo["Comuna"].map(agrupar_comuna)

# ── 4) Foto rápida para entender la distribución (cuenta y proporción)
dist = (
    df_geo["Comuna_grp"]
    .value_counts()
    .to_frame("n")
    .assign(pct=lambda t: (t["n"]/len(df_geo)).round(4))
    .sort_values("n", ascending=False)
)
print("Top categorías en Comuna_grp:")
print(dist.head(15), "\n")

# ── 5) One-hot (dummies) de Comuna agrupada + Región
cols_cat = []
if "Comuna_grp" in df_geo.columns:
    cols_cat.append("Comuna_grp")
if "Region" in df_geo.columns:
    cols_cat.append("Region")

dummies = pd.get_dummies(
    df_geo[cols_cat],
    drop_first=False,      # no colapsamos ninguna; para clustering no hace falta “evitar colinealidad”
    dtype="uint8"
)

print(f"Se generaron {dummies.shape[1]} columnas dummie de {cols_cat}.\nEjemplo de columnas:", dummies.columns[:10].tolist(), "\n")

# ── 6) Guardar intermedios para inspección
Path("../data/interim").mkdir(parents=True, exist_ok=True)
dist.to_csv("../data/interim/geografia_distribucion.csv")
dummies.to_csv("../data/interim/geografia_dummies_v1.csv", index=False)
print("Exportados: ../data/interim/geografia_distribucion.csv y ../data/interim/geografia_dummies_v1.csv")



In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path

df_exp = df.copy()

# 1) Convertimos fechas generales (las que no son ambiguas) con dayfirst=False
fecha_cols = ["Fec_Nac","FechaRegistro","FechaUltimaVisita"]
fecha_cols = [c for c in fecha_cols if c in df_exp.columns]
for c in fecha_cols:
    df_exp[c] = pd.to_datetime(df_exp[c], errors="coerce", dayfirst=False)

# 2) Columnas de planificación: usar dayfirst=True por formato local (DD/MM/AAAA)
hoy = pd.Timestamp(datetime.now().date())
cols_planif = [c for c in ["PrimeraFechaAtencion","FecUltimoPptoCreado"] if c in df_exp.columns]

for c in cols_planif:
    s = pd.to_datetime(df_exp[c], errors="coerce", dayfirst=True)
    df_exp[c] = s
    df_exp[f"{c}_es_planificada"] = s > hoy
    df_exp[f"{c}_dias_hasta"] = (s - hoy).dt.days.where(s > hoy)

# 3) Indicador agregado
plan_cols_flag = [f"{c}_es_planificada" for c in cols_planif]
df_exp["TieneAgendamientoFuturo"] = df_exp[plan_cols_flag].any(axis=1) if plan_cols_flag else False

# 4) Resumen
print("Columnas de planificación evaluadas:", cols_planif)
for c in cols_planif:
    print(f"  {c}_es_planificada ->", df_exp[f"{c}_es_planificada"].mean().round(4), "proporción")
    desc = df_exp[f"{c}_dias_hasta"].describe(percentiles=[.5,.9,.95]).dropna()
    print(f"  {c}_dias_hasta     ->")
    print(desc if not desc.empty else "   (sin futuros)")

print("\nTieneAgendamientoFuturo ->", df_exp["TieneAgendamientoFuturo"].mean().round(4), "proporción total")

# 5) Guardamos copia exploratoria (no subir a Git)
#Path("../data/interim").mkdir(parents=True, exist_ok=True)
#df_exp.to_csv("../data/interim/df_exploracion_paso2.csv", index=False)
#print("\nExportado a ../data/interim/df_exploracion_paso2.csv")


